In [2]:
import time, cv2, sys, os, torch
from threading import Thread
from djitellopy import Tello
import openai
import re

# load environment variables
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/prompts/")

from vqa_question.v1 import prompt_template as vqa_prompt_template
from drone_explore_prompt import prompt_template as explore_template
from drone_next_action.v1 import prompt_template as drone_prompt_template

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/Scripts/")

from data_collection_utils import take_individual_picture, stream_video, stream_frames, record_streamed_frames

print("Imported all modules")


/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imported all modules


In [3]:
from transformers import ViltProcessor, ViltForQuestionAnswering

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [4]:
drone = Tello()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.


In [ ]:
stream = Thread(target=stream_frames, args=(drone, False))
stream.start()

[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'error'


[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'error'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'error'
Exception in thread Thread-8 (stream_frames):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.10/3.10.6_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/homebrew/Cellar/python@3.10/3.10.6_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/sidbendre/project/DroneFormer/Scripts/data_collection_utils.py", line 19, in stream_frames
    drone.streamon()
  File "/opt/homebrew/lib/python3.10/site-packages/djitellopy/enforce_types.py", line 54, in wrapper
    return func(*args, **kwargs)
  File "/opt/homebrew/lib/python3.10/site-packages/djitellopy/tello.py", line 593, in streamon
    self.s

: 

In [4]:
objective = "Find the person who is wearing a blue cap"

In [6]:
drone.takeoff()

[INFO] tello.py - 437 - Send command: 'takeoff'
[WARNING] tello.py - 447 - Aborting command 'takeoff'. Did not receive a response after 20 seconds
[INFO] tello.py - 437 - Send command: 'takeoff'
[WARNING] tello.py - 447 - Aborting command 'takeoff'. Did not receive a response after 20 seconds
[INFO] tello.py - 437 - Send command: 'takeoff'


OSError: [Errno 51] Network is unreachable

In [5]:
drone.streamon()
drone.takeoff()

[INFO] tello.py - 437 - Send command: 'streamon'
[WARNING] tello.py - 447 - Aborting command 'streamon'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'streamon'
[WARNING] tello.py - 447 - Aborting command 'streamon'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'streamon'
[WARNING] tello.py - 447 - Aborting command 'streamon'. Did not receive a response after 7 seconds


Exception: Command 'streamon' was unsuccessful for 4 tries. Latest response:	'Aborting command 'streamon'. Did not receive a response after 7 seconds'

In [6]:
drone.takeoff()

[INFO] tello.py - 437 - Send command: 'takeoff'


KeyboardInterrupt: 

In [32]:
def query_the_environment(drone, question):
    image = drone.get_frame_read().frame
    encoding = processor(image, question, return_tensors="pt")
    outputs = model(**encoding)
    logits = outputs.logits
    idx = torch.sigmoid(logits).argmax(-1).item()
    return str(model.config.id2label[idx])


In [33]:
# Generate next action prompt
def next_action_prompt(prompt_template, objective, environment_context, previous_commands, previous_context, vqa_questions):
    prompt = prompt_template.replace("$objective", objective)
    prompt = prompt.replace("$context", environment_context)
    prompt = prompt.replace("$previous_commands", previous_commands)
    prompt = prompt.replace("$previous_context", previous_context)
    prompt = prompt.replace("$vqa_questions", vqa_questions)
    return prompt, ["\n"] # prompt and stop token

In [34]:
def prompt(prompt, stop_tokens):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.5,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=10,
        stop=stop_tokens,
    )
    return response.choices[0].text

In [35]:
def get_vqa_questions(vqa_prompt_template, objective):
    prompt_text = vqa_prompt_template.replace("$objective", objective)
    result = prompt(prompt_text, ["&&&&&&"])
    matches = re.findall(r'@(.+?)@', result)
    return matches


In [36]:
get_vqa_questions(vqa_prompt_template, objective)

['Is there a person in the image?', 'Is there a blue cap in the image?']

In [ ]:
def explore(drone, use_gpt=False, prompt_template=explore_template):
    if use_gpt:
       prompt(prompt_template)
    else:
        import random as r
        # randomly generate a number between 45 and 180
        random_angle = r.randint(45, 180)
        # randomly generate a number between 5 and 20
        random_distance = r.randint(5, 20)
        actions = [(drone.move_left, random_distance), (drone.move_right, random_distance), (drone.rotate_clockwise, random_angle), (drone.rotate_counter_clockwise, random_angle), (drone.move_forward, random_distance), (drone.move_back, random_distance)]

        # sample 3 actions
        sampled_actions = r.sample(actions, 3)
        for action in sampled_actions:
            action[0](action[1])
            time.sleep(0.5)


In [1]:
# Get the 4 points of a bounding box
def find_corners(upper_left, length: float, width: float):
    corners = []
    
    upper_right = (upper_left[0] + length, upper_left[1])
    lower_left =  (upper_left[0], upper_left[1] + width)
    lower_right = (upper_left[0] + length, upper_left[1] + width)
    
    corners.append(upper_left)
    corners.append(upper_right)
    corners.append(lower_left)
    corners.append(lower_right)
    
    return corners
    
# Find the center of a bounding box
def find_center(corners):
    upper_left, upper_right, lower_left, lower_right = corners
    
    # create diagonal using upper left and lower right
    center_x = (upper_left[0] + upper_right[0]) / 2
    center_y = (upper_left[1] + lower_right[1]) / 2
    
    return(center_x, center_y)


# Exploit Function: Input we have a bounding box with 4 corners (or 1 corner + length and width for a more compact representation)
def exploit(drone, bbox, use_gpt = False, prompt_template = explore_template):
    if use_gpt:
        prompt(prompt_template)
        
    distance_to_object = None
    angle_to_object = None
    height_to_object = None
    actions = [
        (drone.move_left, distance_to_object),
        (drone.move_right, distance_to_object),
        (drone.rotate_clockwise, angle_to_object),
        (drone.rotate_counter_clockwise, angle_to_object),
        (drone.move_forward, distance_to_object),
        (drone.move_back, distance_to_object) 
    ]
        
    # Step 1: Find out whether or not the drone is facing the object
    object_in_frame = False
    while object_in_frame == False:
        drone.rotate_clockwise(30)
    
    
    # Step 2: Translate the object to the center
    bounding_box_center = find_center(bbox)
    current_image = drone.get_frame_read().frame
    height, width, channel = current_image.shape
    frame_center = (width / 2, height / 2)
    
    delta_y = frame_center[1] - bounding_box_center[1]
    delta_x = frame_center[0] - bounding_box_center[0]
    
    if delta_y < 0:
        drone.move_up(abs(delta_y))
    else:
        drone.move_down(abs(delta_y))
    
    if delta_x < 0:
        drone.move_left(abs(delta_x))
    else:
        drone.move_right(abs(delta_x))
    
    # Step 3: Zoom In (move towards the object)
    bbox_length = bbox[1][0] - bbox[0][0]
    bbox_width = bbox[1][1] - bbox[0][1]
    
    while bbox_width <= width or bbox_length <= height:
        drone.move_forward(10)
    
    # Step 4: Terminate
    return

NameError: name 'explore_template' is not defined